In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [3]:
# 시드 고정
import torch
import numpy as np
import random
import os

def my_seed_everywhere(seed: int = 42):
    random.seed(seed) # random
    np.random.seed(seed) # numpy
    os.environ["PYTHONHASHSEED"] = str(seed) # os
    # pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

my_seed = 42
my_seed_everywhere(my_seed)

import tensorflow as tf
import numpy as np
import random
import os

def my_seed_everywhere(seed: int = 42):
    random.seed(seed) # random
    np.random.seed(seed) # np
    os.environ["PYTHONHASHSEED"] = str(seed) # os
    tf.random.set_seed(seed) # tensorflow

my_seed = 42
my_seed_everywhere(my_seed)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tqdm
from transformers import Trainer, TrainingArguments
import re
import pyarrow as pa
from datasets import Dataset
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
from sklearn.model_selection import train_test_split
from datasets import load_metric

In [4]:
dir_path = '/content/drive/MyDrive/AIFFELTHON/TUNiB/선별된 실험 데이터/12.02/'
file_name = 'train_bt_ch_rs100_4000'
data = pd.read_csv(dir_path+file_name+'.csv')
data.head()

,class,label_sentence,non_label_sentence,binary_class,type
0,직장 내 괴롭힘 대화,1:길동 씨 이번에 이것 좀 처리해요\n2:이거 제가 한 게 아닌데요\n1:팀에서 ...,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...,공격 대화,NaN
1,기타 괴롭힘 대화,1:야 너 앞니 왜 그렇게 튀어나왔냐\n2:태어날 때부터 그랬어 물어보지 마\n1:...,야 너 앞니 왜 그렇게 튀어나왔냐\n태어날 때부터 그랬어 물어보지 마\n아 그럼 태...,공격 대화,NaN
2,갈취 대화,1:원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n2:미안 나 오늘은 진짜...,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n미안 나 오늘은 진짜 돈이 ...,공격 대화,NaN
3,갈취 대화,1:너 저번에 술 먹은 날 기억해\n2:아니 왜\n1:야 300만 원만 가져와\n2...,너 저번에 술 먹은 날 기억해\n아니 왜\n야 300만 원만 가져와\n뭔 소리야 ...,공격 대화,NaN
4,협박 대화,1:너 또 내말 무시하냐\n1:이 새끼 널 좆으로 보나 본데\n2:아냐 진짜 시간이...,너 또 내말 무시하냐\n이 새끼 널 좆으로 보나 본데\n아냐 진짜 시간이 없었어 미...,공격 대화,NaN


In [5]:
labels = {'협박 대화':0, '갈취 대화':1, '직장 내 괴롭힘 대화':2, '기타 괴롭힘 대화':3, '일반 대화':4}

num_labels = len(labels)

data['label'] = data['class'].apply(lambda x: labels[x])

model_path = "beomi/KcELECTRA-base-v2022"

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel

HUGGINGFACE_MODEL_PATH = "beomi/KcELECTRA-base-v2022"

tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels=5)

Some weights of the model checkpoint at beomi/KcELECTRA-base-v2022 were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'c

In [6]:
model.config.num_attention_heads = 4

In [7]:
import pyarrow as pa
from datasets import Dataset

data = Dataset(pa.Table.from_pandas(data))

def transform(data):
  return tokenizer(
      data['non_label_sentence'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = True,
      )

encoded_data = data.map(transform, batched=True)

train_dataset, validation_dataset= encoded_data.train_test_split(test_size=0.1).values()

metric = load_metric("f1")

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels, average="micro")

output_dir = '/content/drive/MyDrive/아이펠톤/model_data'

training_arguments = TrainingArguments(
    output_dir, 
    evaluation_strategy="epoch", #evaluation하는 빈도
    learning_rate = 2e-5, 
    per_device_train_batch_size = 8, 
    per_device_eval_batch_size = 8, 
    num_train_epochs = 2,
    weight_decay = 0.01
)


  0%|          | 0/24 [00:00<?, ?ba/s]

<ipython-input-7-a552202b2450>:18: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("f1")


In [ ]:
trainer = Trainer(
    model=model,                           
    args=training_arguments,                  
    train_dataset=train_dataset,    
    eval_dataset=validation_dataset,      
    compute_metrics=compute_metrics
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: class, type, binary_class, non_label_sentence, label_sentence. If class, type, binary_class, non_label_sentence, label_sentence are not expected by `ElectraForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 20892
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5224
  Number of trainable parameters = 127780613


Epoch,Training Loss,Validation Loss


In [ ]:
test_file_path = '/content/drive/MyDrive/아이펠톤/DKTCdata/test.json'
with open(test_file_path, mode='rt', encoding='utf-8') as f:
    test_dataset = pd.read_json(f)

test_data = test_dataset.transpose()
test_X = Dataset(pa.Table.from_pandas(test_data))

def test_transform(data):
  return tokenizer(
      data['text'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = True,
      )

encoded_test_data = test_X.map(test_transform, batched=True)

prediction = trainer.predict(encoded_test_data)

len(prediction.predictions)
pred_label = []

for i in range(len(prediction.predictions)):
    pred_label.append((str)(np.argmax(prediction.predictions[i])).zfill(2))
    
submission = test_data.assign(CLASS=pred_label)
submission = submission.rename(columns={'CLASS':'class'})

submission.drop(['text'], axis=1, inplace=True)
submission = submission.transpose()

import json

model_name = 'KcELECTRA'

submission_file_path = '/content/drive/MyDrive/아이펠톤/sub/' + 'sub_' + model_name + file_name[5:] + '-9' + '.json'
result = submission.to_json(submission_file_path)

with open(submission_file_path) as f:
    parsed = json.load(f)

with open(submission_file_path, 'w') as f:
    json.dump(parsed, f, indent=4)
